In [1]:
import numpy as np
import xarray as xr
import sys
sys.path.insert(1, '../functions/')
from statistics_functions import *
from pyproj import Proj

In [2]:
res = '1km'

path_nc = f'/p/scratch/chhb19/gaertner2/interpolated_fesom_output/daily/{res}/'
path_loc = '/p/project/chhb19/gaertner2/data/awicm_cvmix/'

In [3]:
def mask_arctic_basin(var):
    
    # generate Arctic Basin mask
    mask = ((((lon > -120) & (lon < 100)) & (lat >= 80)) |
            ((lon <= -120) & (lat >= 70)) |
            ((lon >= 100) & (lat >= 70)))
    index_x = np.where(np.sum(mask[1:-1,1:-1],axis=0)>0)
    index_y = np.where(np.sum(mask[1:-1,1:-1],axis=1)>0)
    
    # shrink field and apply mask
    if len(np.shape(var))==3:
        var = var[:,1:-1, 1:-1]
        var = np.where(mask[1:-1,1:-1], var, np.nan)
        var = var[:,
                  max([0,index_y[0][0]-1]):index_y[0][-1]+2,
                  max([0,index_x[0][0]-1]):index_x[0][-1]+2]
    else:
        var = var[1:-1, 1:-1]
        var = np.where(mask[1:-1,1:-1], var, np.nan)
        var = var[max([0,index_y[0][0]-1]):index_y[0][-1]+2,
                  max([0,index_x[0][0]-1]):index_x[0][-1]+2]
    
    return var

In [4]:
# calculate spatial resolution
file = xr.open_dataset(path_nc + f'2013_{res}.nc')

lon = file.ULON
lat = file.ULAT

m = Proj(proj='stere',lat_0=90, lat_ts=70, lon_0=-45, ellps='WGS84')

x, y = m(lon, lat)
dxu = np.sqrt((x[:,1:]-x[:,:-1])**2 + (y[:,1:]-y[:,:-1])**2)
dxu = np.concatenate([dxu,dxu[:,-1].reshape((dxu.shape[0],1))],axis=1)
dyu = np.sqrt((x[1:,:]-x[:-1,:])**2 + (y[1:,:]-y[:-1,:])**2)
dyu = np.concatenate([dyu,dyu[-1,:].reshape((1,dyu.shape[1]))],axis=0)

dxu = mask_arctic_basin(dxu)
dyu = mask_arctic_basin(dyu)

res_km = 0.5 * (np.nanmean(dxu) + np.nanmean(dyu)) / 1000

In [5]:
years = [i for i in range(2013, 2021)]
years += [i for i in range(2093, 2101)]

In [6]:
[aice_mean, area_total, hice_mean, vol_total] = [[] for _ in range(4)]
for year in years:
    file = xr.open_dataset(path_nc + f'{year}_{res}.nc')
    
    aice = mask_arctic_basin(file.A)
    aice_mean += np.nanmean(aice, axis=(1,2)),
    area_total += np.nansum(aice * dxu * dyu, axis=(1,2)) / 1e6, # in km2
    
    hice = mask_arctic_basin(file.H)
    hice_mean += np.nanmean(hice, axis=(1,2)),
    vol_total += np.nansum(hice * dxu * dyu, axis=(1,2)) / 1e12, # in 1000 km3 

In [7]:
np.save(path_loc + f'ice_area_thickness_{res}.npy',
        np.array([aice_mean, area_total, hice_mean, vol_total, years], dtype='object'))